**NEURAL NETWORKS FROM SCRATCH**

Some utility functions

In [17]:
import numpy as np

import os

os.environ['OMP_NUM_THREADS'] = '11'

def tanh(x):
    return np.tanh(x);

def tanh_prime(x):
    return 1-np.tanh(x)**2;

def mse(y_true, y_pred):
    return np.mean(np.power(y_true-y_pred, 2));

def mse_prime(y_true, y_pred):
    return 2*(y_pred-y_true)/y_true.size;

def relu(x):
    return np.maximum(0, x)

def relu_prime(x):
    return np.where(x > 0, 1, 0)

Abstract class Layer

In [18]:
class Layer:
    def __init__(self):
        self.input = None
        self.output = None

    def forward_propagation(self, input):
        raise NotImplementedError

    def backward_propagation(self, output_error, learning_rate):
        raise NotImplementedError

Fully-connected layer

In [19]:
class FCLayer(Layer):
    def __init__(self, input_size, output_size):
        self.weights = np.random.rand(input_size, output_size) - 0.5
        self.bias = np.random.rand(1, output_size) - 0.5

    def forward_propagation(self, input_data):
        self.input = input_data
        self.output = np.dot(self.input, self.weights) + self.bias
        return self.output

    def backward_propagation(self, output_error, learning_rate):
        input_error = np.dot(output_error, self.weights.T)
        weights_error = np.dot(self.input.T, output_error)
        self.weights -= learning_rate * weights_error
        self.bias -= learning_rate * output_error
        return input_error

Droput Layer

In [20]:
class DropoutLayer(Layer):
    def __init__(self, p):
        self.p = p
        self.mask = None

    def forward_propagation(self, input_data):
        self.mask = np.random.rand(*input_data.shape) > self.p
        self.output = input_data * self.mask / (1 - self.p)
        return self.output

    def backward_propagation(self, output_error, learning_rate):
        return output_error * self.mask / (1 - self.p)

Activation layer

In [21]:
class ActivationLayer(Layer):
    def __init__(self, activation, activation_prime):
        self.activation = activation
        self.activation_prime = activation_prime

    def forward_propagation(self, input_data):
        self.input = input_data
        self.output = self.activation(self.input)
        return self.output

    def backward_propagation(self, output_error, learning_rate):
        return self.activation_prime(self.input) * output_error
    
    def forward_propagation_dropout(self, input_data):
        self.input = input_data
        self.output = self.activation(self.input)
        return self.output

    def backward_propagation_dropout(self, output_error, learning_rate):
        return self.activation_prime(self.input) * output_error

Network class

In [22]:
class Network:
    def __init__(self):
        self.layers = []
        self.loss = None
        self.loss_prime = None

    def add(self, layer):
        self.layers.append(layer)

    def use(self, loss, loss_prime):
        self.loss = loss
        self.loss_prime = loss_prime

    def predict(self, input_data):
        samples = len(input_data)
        result = []
        for i in range(samples):
            output = input_data[i]
            for layer in self.layers:
                output = layer.forward_propagation(output)
            result.append(output)
        return result

    def fit(self, x_train, y_train, epochs, learning_rate):
        samples = len(x_train)
        for i in range(epochs):
            err = 0
            for j in range(samples):
                output = x_train[j]
                for layer in self.layers:
                    output = layer.forward_propagation(output)
                err += self.loss(y_train[j], output)
                error = self.loss_prime(y_train[j], output)
                for layer in reversed(self.layers):
                    error = layer.backward_propagation(error, learning_rate)
            err /= samples
            print('epoch %d/%d   error=%f' % (i+1, epochs, err))

Solve the XOR problem ReLU

In [11]:
x_train = np.array([[[0,0]], [[0,1]], [[1,0]], [[1,1]]])
y_train = np.array([[[0]], [[1]], [[1]], [[0]]])
np.random.seed(10)
net = Network()
net.add(FCLayer(2, 10))
net.add(ActivationLayer(relu, relu_prime))  # Use ReLU activation
net.add(DropoutLayer(p=0.1))  # Add dropout layer
net.add(FCLayer(10, 1))
net.add(ActivationLayer(relu, relu_prime))
net.use(mse, mse_prime)
net.fit(x_train, y_train, epochs=500, learning_rate=0.1)

epoch 1/500   error=0.408052
epoch 2/500   error=0.370493
epoch 3/500   error=0.317765
epoch 4/500   error=0.354062
epoch 5/500   error=0.208899
epoch 6/500   error=0.330154
epoch 7/500   error=0.325149
epoch 8/500   error=0.271991
epoch 9/500   error=0.260267
epoch 10/500   error=0.270214
epoch 11/500   error=0.253238
epoch 12/500   error=0.386419
epoch 13/500   error=0.297254
epoch 14/500   error=0.269202
epoch 15/500   error=0.318728
epoch 16/500   error=0.257841
epoch 17/500   error=0.266896
epoch 18/500   error=0.254251
epoch 19/500   error=0.250129
epoch 20/500   error=0.250991
epoch 21/500   error=0.259996
epoch 22/500   error=0.250001
epoch 23/500   error=0.250000
epoch 24/500   error=0.255236
epoch 25/500   error=0.250111
epoch 26/500   error=0.250000
epoch 27/500   error=0.253675
epoch 28/500   error=0.334622
epoch 29/500   error=0.250145
epoch 30/500   error=0.269934
epoch 31/500   error=0.250735
epoch 32/500   error=0.252057
epoch 33/500   error=0.270597
epoch 34/500   erro

Solve the MNIST problem ReLU

In [12]:
from keras.datasets import mnist
from keras.utils import to_categorical
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(x_train.shape[0], 1, 28*28)
x_train = x_train.astype('float32')
x_train /= 255
y_train = to_categorical(y_train)
x_test = x_test.reshape(x_test.shape[0], 1, 28*28)
x_test = x_test.astype('float32')
x_test /= 255
y_test = to_categorical(y_test)
np.random.seed(10)
net = Network()
net.add(FCLayer(28*28, 100))
net.add(ActivationLayer(relu, relu_prime))  # Use ReLU activation
net.add(FCLayer(100, 50))
net.add(ActivationLayer(relu, relu_prime))
net.add(DropoutLayer(p=0.1))  # Add dropout layer# Use ReLU activation
net.add(FCLayer(50, 10))
net.add(ActivationLayer(relu, relu_prime))

net.use(mse, mse_prime)
net.fit(x_train, y_train, epochs=20, learning_rate=0.1)
out = net.predict(x_test[0:10])
print("\n")
print("predicted values : ")
print(out, end="\n")
print("true values : ")
print(y_test[0:10])

epoch 1/20   error=3947886691824435468380254489894103931519315739786260544691370348873126208501448704.000000
epoch 2/20   error=0.100000
epoch 3/20   error=0.100000
epoch 4/20   error=0.100000
epoch 5/20   error=0.100000
epoch 6/20   error=0.100000
epoch 7/20   error=0.100000
epoch 8/20   error=0.100000
epoch 9/20   error=0.100000
epoch 10/20   error=0.100000
epoch 11/20   error=0.100000
epoch 12/20   error=0.100000
epoch 13/20   error=0.100000
epoch 14/20   error=0.100000
epoch 15/20   error=0.100000
epoch 16/20   error=0.100000
epoch 17/20   error=0.100000
epoch 18/20   error=0.100000
epoch 19/20   error=0.100000
epoch 20/20   error=0.100000


predicted values : 
[array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]), array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]), array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]), array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]), array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]), array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]), array([[0., 0., 0.,

Regular XOR

In [26]:
x_train = np.array([[[0,0]], [[0,1]], [[1,0]], [[1,1]]])
y_train = np.array([[[0]], [[1]], [[1]], [[0]]])
np.random.seed(10)
net = Network()
net.add(FCLayer(2, 10))
net.add(ActivationLayer(tanh, tanh_prime))
net.add(FCLayer(10, 3))
net.add(ActivationLayer(tanh, tanh_prime))
net.use(mse, mse_prime)
net.fit(x_train, y_train, epochs=500, learning_rate=0.1)
out = net.predict(x_train)
print(out)

epoch 1/500   error=0.399597
epoch 2/500   error=0.346825
epoch 3/500   error=0.336379
epoch 4/500   error=0.331577
epoch 5/500   error=0.327561
epoch 6/500   error=0.323962
epoch 7/500   error=0.320659
epoch 8/500   error=0.317585
epoch 9/500   error=0.314694
epoch 10/500   error=0.311952
epoch 11/500   error=0.309332
epoch 12/500   error=0.306812
epoch 13/500   error=0.304373
epoch 14/500   error=0.302000
epoch 15/500   error=0.299679
epoch 16/500   error=0.297398
epoch 17/500   error=0.295148
epoch 18/500   error=0.292918
epoch 19/500   error=0.290699
epoch 20/500   error=0.288483
epoch 21/500   error=0.286261
epoch 22/500   error=0.284026
epoch 23/500   error=0.281771
epoch 24/500   error=0.279487
epoch 25/500   error=0.277167
epoch 26/500   error=0.274806
epoch 27/500   error=0.272398
epoch 28/500   error=0.269937
epoch 29/500   error=0.267420
epoch 30/500   error=0.264847
epoch 31/500   error=0.262215
epoch 32/500   error=0.259529
epoch 33/500   error=0.256794
epoch 34/500   erro

Regular MNisT

In [24]:
from keras.datasets import mnist
from keras.utils import to_categorical
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(x_train.shape[0], 1, 28*28)
x_train = x_train.astype('float32')
x_train /= 255
y_train = to_categorical(y_train)
x_test = x_test.reshape(x_test.shape[0], 1, 28*28)
x_test = x_test.astype('float32')
x_test /= 255
y_test = to_categorical(y_test)
np.random.seed(10)
net = Network()
net.add(FCLayer(28*28, 100))
net.add(ActivationLayer(tanh, tanh_prime))
net.add(FCLayer(100, 50))
net.add(ActivationLayer(tanh, tanh_prime))
net.add(FCLayer(50, 10))
net.add(ActivationLayer(tanh, tanh_prime))
net.use(mse, mse_prime)
net.fit(x_train, y_train, epochs=20, learning_rate=0.1)
out = net.predict(x_test[0:10])
print("\n")
print("predicted values : ")
print(out, end="\n")
print("true values : ")
print(y_test[0:10])

epoch 1/20   error=0.042247
epoch 2/20   error=0.021373
epoch 3/20   error=0.017366
epoch 4/20   error=0.015063
epoch 5/20   error=0.013448
epoch 6/20   error=0.012274
epoch 7/20   error=0.011323
epoch 8/20   error=0.010582
epoch 9/20   error=0.009878
epoch 10/20   error=0.009291
epoch 11/20   error=0.008769
epoch 12/20   error=0.008260
epoch 13/20   error=0.007880
epoch 14/20   error=0.007516
epoch 15/20   error=0.007212
epoch 16/20   error=0.006885
epoch 17/20   error=0.006660
epoch 18/20   error=0.006405
epoch 19/20   error=0.006194
epoch 20/20   error=0.005981


predicted values : 
[array([[ 4.89452702e-03, -1.24184322e-03, -1.58183250e-02,
         3.14040731e-03,  4.40748188e-02, -3.84802814e-03,
         2.44695914e-04,  9.89555274e-01, -9.52175132e-03,
         1.25411090e-02]]), array([[0.02892183, 0.0058073 , 0.96615312, 0.03146359, 0.01658489,
        0.04972078, 0.00800865, 0.01891473, 0.06065277, 0.02108738]]), array([[ 8.24568224e-04,  9.93761057e-01, -4.10738017e-03,
   